# (Weighted) Kemeny Rank Aggregation

In [2]:
# Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
# SPDX-License-Identifier: Apache-2.0

import numpy as np

import scipy as sp
import pandas as pd
from tqdm import tqdm, trange
import os
from sklearn.model_selection import ParameterGrid
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import ndcg_score
from scipy.stats import kendalltau, spearmanr
import matplotlib.pyplot as plt

from __future__ import print_function
import numpy as np
from itertools import combinations, permutations
from typing import Optional
import pickle as pkl
from joblib import Parallel, delayed

from distributions import mallows_kendall as mk
from distributions import permutil as pu
from distributions import sampling


### Experiment 1: Impact of Noisy Rankings on Kemeny Rank Aggregation

In [ ]:
parameter_grid = {
'N' : [10, 50], # Number of permutations
'n' : [50, 100], # Number of items
'theta': [0.05, 0.1, 0.2],
'type': ['random', 'mixture'],
'scale': [100],
'noise': [0.0, 0.1, 0.25, 0.5],
'seed': [0, 1, 2],
}

results = {}
EXP_NUM = 1
for parameters in tqdm(list(ParameterGrid(parameter_grid))):
    ranks, _, _ = sampling.sample_mallows_with_noise(**parameters)
    N, n = ranks.shape
    true_rank = np.arange(n)
    _, aggregated_rank = rank_aggregation.borda(ranks)
    
    results[str(EXP_NUM)] = {
        'N' : parameters['N'],
        'n' : parameters['n'],
        'theta': parameters['theta'],
        'type': parameters['type'],
        'scale': parameters['scale'],
        'noise': parameters['noise'],
        'seed': parameters['seed'],
        'Distance of Median from True Rank': mk.distance(aggregated_rank),
        'NDCG of Median w.r.t. True Rank': ndcg_score(true_rank.reshape((1, -1)), aggregated_rank.reshape((1, -1))),
    }
    EXP_NUM = EXP_NUM + 1

results = pd.DataFrame(results).T
results.to_csv('experiments/impact_of_noise.csv', index=None)

# View the results
results = pd.read_csv('experiments/impact_of_noise.csv', index_col=0)
results

In [ ]:
# View some cases
filtered_result = results.loc[(results['N'] == 50) & (results['n'] == 100) & (results['theta'] == 0.1)]

for noise in [0.0, 0.1, 0.25, 0.5]:
    print(f'Noise: {noise}')
    print(filtered_result.loc[filtered_result['noise'] == noise].iloc[:, -2:].mean())

In [ ]:
import matplotlib.pyplot as plt
params = {'text.usetex' : False,
          'font.size' : 16,
          "font.family": "serif",
          "font.serif": ["Palatino"],
          }
plt.rcParams.update(params)

noise = [str(i) for i in [0.0, 0.1, 0.25, 0.5]]
fig, axes = plt.subplots(1, 3, figsize=(20, 5), sharey=True, sharex=True)
axes[0].bar(x=noise, height=[46.66, 46.66, 149.00, 326.16], width=0.5, color='darkblue')
axes[0].set_title(r"N = 50, n = 100, \theta = 0.2", fontsize=16)
axes[0].set_ylabel("Distance of Median from True Rank", fontsize=16)
axes[0].set_xlabel("Percentage of Noisy Samples", fontsize=16)

axes[1].bar(x=noise, height=[116.00, 151.00, 211.00, 396.66], width=0.5, color='darkblue')
axes[1].set_title(r"N = 50, n = 100, \theta = 0.1", fontsize=16)
axes[1].set_xlabel("Percentage of Noisy Samples", fontsize=16)

axes[2].bar(x=noise, height=[247.33, 283.16, 346.50, 581.66], width=0.5, color='darkblue')
axes[2].set_title(r"N = 50, n = 100, \theta = 0.05", fontsize=16)
axes[2].set_xlabel("Percentage of Noisy Samples", fontsize=16)

# fig.savefig('test.png')

## Experiment
In models with strong concensus, the reliability of a metric is negatively correlated with the objective value i.e. if we remove a point which occurs under the model with low probability, then the objective results in a closely clustered value.  

In [ ]:
parameter_grid = {
'm' : [10, 50], # Number of metrics
'n' : [50, 100], # Number of models
'theta' : [0.01, 0.1, 0.2]
}

ranking_data = {}
std_dist_from_true_rank = []
dist_of_median_from_true_rank = []
for parameters in tqdm(list(ParameterGrid(parameter_grid))):
    true_rank = np.arange(parameters['n'])
    ranks = mk.sample(**parameters)
    n_metrics, n_models = ranks.shape

    objective_values = []
    true_rank = np.arange(n_models)
    distance_from_true_rank = [mk.distance(ranks[i, :], true_rank) for i in range(n_metrics-1, -1, -1)]
    for idxs in combinations(np.arange(n_metrics), n_metrics-1):
        borda_agg = mk.median(ranks[idxs, :])
        objective_values.append(objective(borda_agg, ranks[idxs, :])/(n_metrics-1))
    
    print(kendalltau(objective_values, distance_from_true_rank))
    print(spearmanr(objective_values, distance_from_true_rank))
    
    plt.scatter(objective_values, distance_from_true_rank)
    plt.title(f"# models = {n_models} # metrics = {n_metrics} theta = {parameters['theta']}")
    plt.xlabel('Average distance of median from samples')
    plt.ylabel('Distance of excluded sample from true rank')
    plt.show()

### Identifying the Noisy Permutations

X-axis (Average distance of median from samples) denotes the objective of the insample points i.e. average distance of the insample points from their median. 
Y-axis (Distance of excluded sample from true rank) denotes the average distance of the outsample points from the true rank (identity permutation). 

The highly negative correlation implies that excluding low-probability (outlying) points, improves the "tightness" of the inlying points. 

In [ ]:
parameter_grid = {
'm' : [10, 50], # Number of metrics
'n' : [50, 100], # Number of models
'theta' : [0.001, 0.01, 0.1]
}

NOISE = 0.1
TOP_K = 0.95
ranking_data = {}
std_dist_from_true_rank = []
dist_of_median_from_true_rank = []
for parameters in tqdm(list(ParameterGrid(parameter_grid))):
    true_rank = np.arange(parameters['n'])
    ranks_mallow = mk.sample(**parameters)
    ranks_random = sample_random(m=int(NOISE*parameters['m']), n=parameters['n'])
    ranks = np.concatenate([ranks_mallow, ranks_random], axis=0)
    
    n_metrics, n_models = ranks.shape

    objective_values = []
    average_distance_of_outsample = []
    true_rank = np.arange(n_models)
    distance_from_true_rank = np.array([mk.distance(perm, true_rank) for perm in ranks])
    n_insample_metrics = int(TOP_K*n_metrics)
    
    for idxs in combinations(np.arange(n_metrics), n_insample_metrics):
        idxs_complement = np.setdiff1d(np.arange(n_metrics), idxs)
        borda_agg = mk.median(ranks[idxs, :])
        objective_values.append(objective(borda_agg, ranks[idxs, :])/(n_insample_metrics))
        average_distance_of_outsample.append(np.mean(distance_from_true_rank[idxs_complement]))
    
    print(kendalltau(objective_values, average_distance_of_outsample))
    print(spearmanr(objective_values, average_distance_of_outsample))
    
    plt.scatter(objective_values, average_distance_of_outsample)
    plt.title(f"# models = {n_models} # metrics = {n_metrics} theta = {parameters['theta']}")
    plt.xlabel('Average distance of median from samples')
    plt.ylabel('Distance of excluded sample from true rank')
    plt.show()

## Measures of Reliability versus True Reliability

In [ ]:
from distibutions.sampling import sample_mallows_with_noise
from model_selection.rank_aggregation import borda, kemeny, trimmed_borda, trimmed_kemeny, influence, pagerank, proximity

In [ ]:
parameters = {
    'N' : 50, # Number of permutations
    'n' : 100, # Number of items
    'theta' : 0.2,
    'noise': 0.5,
    'type': 'mixture', 
    'seed': 0
    } 

ranks, n_noisy, n_mallows = sample_mallows_with_noise(**parameters)
N, n = ranks.shape
true_rank = np.arange(n)

_, borda_rank = borda(ranks, weights=None)
borda_rank_dist = mk.distance(borda_rank)
# print(borda_rank_dist)

dist_from_true_rank = np.array([mk.distance(r) for r in ranks])
# print(dist_from_true_rank)

# Red -- Least probable data points
# Blue -- Most probable data points
colors = n_mallows*['darkblue']
colors.extend(n_noisy*['red'])

influence_of_ranks = influence(ranks, aggregation_type='borda')
proximity_of_ranks = proximity(ranks)
pagerank_of_ranks = pagerank(ranks)
# print(influence_of_ranks, proximity_of_ranks, pagerank_of_ranks)
plt.style.use('ggplot')
figure, axis = plt.subplots(1, 3, figsize=(16, 4), sharey=True)
figure.suptitle(f"Anomaly Type: {parameters['type']}   Theta: {parameters['theta']}   Noise : {parameters['noise']}", fontsize=16)
axis[0].scatter(influence_of_ranks, dist_from_true_rank, c=colors, alpha=0.7)
axis[0].set_xlabel('Influence of Permutation', fontsize=12)
axis[0].set_ylabel('Distance from Central Permutation', fontsize=12)
axis[1].scatter(proximity_of_ranks, dist_from_true_rank, c=colors, alpha=0.7)
axis[1].set_xlabel('Proximity of Permutation', fontsize=12)
# axis[1].set_ylabel('Distance from Central Permutation', fontsize=12)
axis[2].scatter(pagerank_of_ranks, dist_from_true_rank, c=colors, alpha=0.7)
axis[2].set_xlabel('Pagerank of Permutation', fontsize=12)
# axis[2].set_ylabel('Distance from Central Permutation', fontsize=12)

### Improvement due to Trimmed Ranking

In [ ]:
parameters = {
    'N' : 50, # Number of permutations
    'n' : 50, # Number of items
    'theta' : 0.2,
    'noise': 0.50,
    'type': 'mixture', 
    'seed': 0
    } 

TOP_K = int(0.5*parameters['N'])

ranks, n_noisy, n_mallows = sample_mallows_with_noise(**parameters)
N, n = ranks.shape
true_rank = np.arange(n)

_, borda_rank = borda(ranks, weights=None)
borda_rank_dist = mk.distance(borda_rank)
# print(borda_rank_dist)

dist_from_true_rank = np.array([mk.distance(r) for r in ranks])
# print(dist_from_true_rank)

# Red -- Least probable data points
# Blue -- Most probable data points
colors = n_mallows*['darkblue']
colors.extend(n_noisy*['red'])

_, influence_trimmed_borda_rank = trimmed_borda(ranks, weights=None, top_k=TOP_K, aggregation_type='borda', metric='influence')
influence_trimmed_borda_rank_dist = mk.distance(influence_trimmed_borda_rank)
_, proximity_trimmed_borda_rank = trimmed_borda(ranks, weights=None, top_k=TOP_K, aggregation_type='borda', metric='proximity')
proximity_trimmed_borda_rank_dist = mk.distance(proximity_trimmed_borda_rank)
_, pagerank_trimmed_borda_rank = trimmed_borda(ranks, weights=None, top_k=TOP_K, aggregation_type='borda', metric='pagerank')
pagerank_trimmed_borda_rank_dist = mk.distance(pagerank_trimmed_borda_rank)

# print(influence_of_ranks, proximity_of_ranks, pagerank_of_ranks)
print('Improvement in Distance from Central Permutation post Trimming')
print('Influence', borda_rank_dist - influence_trimmed_borda_rank_dist)
print('Proximity', borda_rank_dist - proximity_trimmed_borda_rank_dist)
print('Pagerank', borda_rank_dist - pagerank_trimmed_borda_rank_dist)